In [1]:
from bokeh.plotting import output_notebook
output_notebook()

Loading BokehJS ...

In [2]:
from bokeh.plotting import figure, show

In [3]:
import pandas as pd

df = pd.read_csv("data/dose_response.csv", index_col=0)

In [4]:
df.head()

,PC1,PC2,PC3,PC4,PC5,PC6
Control,15198.737,20976.048,14439.106,18371.778,24103.563,24641.569
100,4048.306,4699.495,5166.628,5064.056,7206.184,3583.669
50,4954.478,6235.781,6308.402,7117.099,7667.698,4732.384
20,6370.060,8784.412,9188.958,14872.448,17572.788,10107.329
5,17354.263,18351.751,20841.108,20500.017,26013.620,18634.733


In [31]:
def hill_equation(x, a, b, c, d):
    y = a + (b-a)/(1+(c/x)**d)
    return y

In [33]:
def dose_response(df):
    """ 
    Dataframe should be in the format rows = concentrations, columns = replicates
    The first row sould be the control
    
    """
    from bokeh.plotting import figure
    import numpy as np
    import math
    from bokeh.models import FixedTicker
    
    from scipy.optimize import curve_fit
    color ='blue'
    alpha = 0.6
    control = df.iloc[0, :]
    tests = df.iloc[1:, :]
    
    points = (tests / control.mean() * 100).mean(1)
    std = (tests / control.mean() * 100).std(1)
    err_y = list(zip(points + std, points - std))
    
    x_labels = list(points.index)
    f = figure()
    f.circle(x=x_labels, y=points.values, color=color, alpha=alpha)
    f.multi_line(xs=list(zip(x_labels, x_labels)), 
                 ys=err_y, color=color, alpha=alpha)

    popt, pcov = curve_fit(hill_equation, 
                           tests.index.astype(float).tolist(), 
                           (tests / control.mean() * 100).mean(1).values
                          )
    

    x = np.linspace(tests.index.astype(float).max(), tests.index.astype(float).min(), num=5000)
    y = hill_equation(x, *popt)

    f.line(x, y, color=color, alpha=alpha, line_dash='dashed')
    
    
    x_labels = tests.index.astype(float).tolist()
    
    f.xaxis.ticker=FixedTicker(ticks=sorted(x_labels))

    
    #lc50 = popt[2]
    #f.cross(lc50, 50, color='red', legend='LC50')
    return f
    
plot = dose_response(df)
show(plot)